In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn import datasets

In [ ]:
boston_data=datasets.load_boston()

In [ ]:
boston_df=pd.DataFrame(boston_data.data, columns= boston_data.feature_names)

In [ ]:
boston_df['medv']= boston_data.target

### The Best Subset Selection method

We studied this method in the "Statistics with R class" as a way of choosing a good multiple regression equation. 

Remember that choosing between equations with different number of predictor variables should be done with care. The best subset selection method allows us to navigate this problem and get a good solution.

In R, we used the "regsubsets()" method from the "leaps" library to apply the best subset selection method.

The "regsubsets()" method returns a list with the best equation of each size (the best equation with one predictor, the best equation with two predictors,...., and the best equation  with p predictors).

Then, the "regsubsets()" method allows us to compare these equations and choose the single best one. This comparison can be done using adjusted R squared, Cp, or BIC (Bayesian Information Criterion). In our case, we used adjusted R squared to do these comparisons.

### The Best Subset Selection method in Python

In Python, we can run a best subset selection analysis by using the methods from the ABESS library.

ABESS: Adaptative Best Subset Selection

ABESS uses a rather advanced algorithm to run best subset selection; thus, it is beyond the scope of this class to go over the details of this algorithm. We will focus on learning __how to apply ABESS in Python__ and __how to interpret its results.__

ABESS uses BIC (Bayesian Information Criterion) to choose the best model. 

In [ ]:
from abess import abessLm

In [ ]:
# from abess import LinearRegression

In [ ]:
model_abess = abessLm(support_size = range(14))  # range(14) equivalent to np.arange(14)

In [ ]:
# model_abess = LinearRegression(support_size = range(14))

The argument __support size__ is similar to the argument __mvmax__ in R. It represents the maximum model size that you want to consider. In this case, we want to consider all possible model sizes, from 1 ( a model with one predictor) all the way to 13 predictors (a model with all predictors).

Now we need to call the method __fit()__ on the object _model_abess_ that we created in the previous cell.

The fit() method takes arrays as arguments. Thus, we need to convert the boston data from a Pandas data frame into a Numpy array.

The method fit () takes two arguments: X (the array with the predictor values) and Y (the array with the values of Y)

In [ ]:
boston_df.head()

In [ ]:
# Creating an array with the predictors
# Convert the boston_df, excluding the last column which is 'medv', into a Numpy array

Xs_array= np.array(boston_df.iloc[:,:-1])

In [ ]:
# Creating an array with Y

y_array= np.array(boston_df['medv'])

In [ ]:
model_abess.fit(Xs_array, y_array)

The result of the ABESS algorithm is an array of coefficients where __only the coefficients for the predictors that ABESS has chosen has a value different from zero__. The predictors that ABESS excludes from the chosen equation have a coefficient of zero.

We can call the property "coef_" on the object _model_abess_ that we created before to see what coefficients are different from zero.

In [ ]:
model_abess.coef_

If we want to select only the coefficients different from zero, we can take the following steps:

In [ ]:
model_abess.coef_!=0

We can store this Boolean array in a variable to use it later to index the array of coefficients

In [ ]:
non_zero_indexes = model_abess.coef_!=0
non_zero_indexes

In [ ]:
# Only get the coefficients different from zero

model_abess.coef_[non_zero_indexes]

What are the predictors which coefficients are different from zero? In other words, what are the predictors chosen by ABESS?

In [ ]:
# Get a list with the column names for the predictors

boston_df.iloc[:,:-1].columns

In [ ]:
 # Get the predictors which coefficients are different from zero

boston_df.iloc[:,:-1].columns[non_zero_indexes]

In [ ]:
selected_predictors= boston_df.iloc[:,:-1].columns[non_zero_indexes]
selected_predictors

In [ ]:
print ('According to Abess, the predictors to be included in the model are :', selected_predictors)

In [ ]:
print ('According to Abess, the predictors to be included in the model are :', list(selected_predictors))

In [ ]:
pd.DataFrame({'Coefficients':model_abess.coef_[non_zero_indexes]}, index=list(selected_predictors))

#### Last comments about ABESS

- When you apply the best subset selection method in R to return the coefficients of the model with seven predictors you get the same coefficients that we just got! This was expected, but it is good to check that results are consistent across tools.


- One problem with the previous list of coefficients is that it does not include the intercept. One solution could be to run linear regression using Statsmodels as we did in a previous session. 

You would run the regression in Statsmodels with these seven predictors. The output of Statsmodels will return the intercept and these seven coefficients.

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
smf.ols('medv~'+ '+'.join(selected_predictors), data=boston_df).fit().params

In [ ]:
smf.ols('medv~'+ '+'.join(selected_predictors), data=boston_df).fit().summary()

### More on the best subset selection method

Now, let's try to code some of the steps involved in the best subset selection algorithm.

We will only code, __in a very intuitive and raw way__, some of the steps needed to conduct the first task required by the method: select the best model for each size.

One of the first things we can do is to write a function that estimates a regression model and its corresponding R squared given:

a) An array of Y values

b) The two-dim array which columns contain the values for all the predictors

c) A list of the predictors that the user wants to include in the model

In [ ]:
# Here I chose to use 'statsmodels.api' instead of 'statsmodels.formula.api' because the former is simpler when it comes
# to write more generic code (i.e., code to include in a function)

import statsmodels.api as sm

In [ ]:
def my_reg_model_rsq (y,Xs, feature_set):
    
    """y and Xs are array-like objects.
    They could be arrays or data frames.
    Xs is the matrix with all the predictors.
    y is the array/Series with the dependent variable values.
    feature_set must be a list with the names of the predictors to incude in the equation
    """
    X = sm.add_constant(Xs[feature_set])
    reg_model= sm.OLS(y,X).fit()
    return(reg_model.rsquared)

__Note__: The following commands and the loops afterwards should all be included in a function that can be more generic and reusable. However, let's keep things simpler by hard-coding some the logic involved in this problem.

In [ ]:
predictors_list= list(boston_df.iloc[:,:-1].columns)

Task # 1 in best subset selection is to get the best equation of each size. That is, the best equation with 1 predictor, the best equation with two predictors,..., the best equation with p predictors.

Task # 2: Select the single best among the previous equations. We based this comparison on Adj R squared, BIC, Cp, test MSE (via CV), or other suitable metric.

Task # 1:

Step 1: Select the best equation with one predictor (the one with the highest R squared)

In [ ]:
rsq_list=[]
for i in predictors_list:
    # In the next line, there is a call to the function 'my_reg_model_rsq' that we created earlier
    rsq_list.append(my_reg_model_rsq(boston_df['medv'],boston_df.iloc[:,:-1],i))

In [ ]:
print('The best equation with one predictor is the one that includes :', np.array(predictors_list)[rsq_list==max(rsq_list)])
print('The Rsq of this equation is: ', np.round(max(rsq_list),2))

Task # 1:

Step 2: Select the best equation with two predictors

Let's repeat the previous logic to get the best equation with two predictors.

We need to figure out how to get __a list with all the combinations of two predictors__ that can formed from the 13 predictors available in the Boston dataset.

The method combinations() from the itertools library can be used with this purpose.

In [ ]:
import itertools

In [ ]:
list_two_predictors= list(itertools.combinations(predictors_list, 2))

In [ ]:
list_two_predictors

In [ ]:
rsq_list1=[]
for i in list_two_predictors:
    rsq_list1.append(my_reg_model_selection(boston_df['medv'],boston_df.iloc[:,:-1],list(i))) 
    # list(i) instead of i because i is a tuple and my_reg_model_selection takes a list as the 3rd argument

In [ ]:
print('The best equation with two predictors is the one that includes :', np.array(list_two_predictors)[rsq_list1==max(rsq_list1)])
print('The Rsq of this equation is: ', np.round(max(rsq_list1),2))

### Work independently on the following tasks

### You will have five minutes to work on this

Task 1: Repeat the previous logic and get the best equation with three predictors

Task 2: Apply ABESS to get the best equation with three predictors and check that you get the same result obtained in task 1